Groupby, Pivot, Merge

In [1]:
import pandas as pd
import numpy as np

univ = ['AAPl','MSFT','BAC','GS']
dates = pd.date_range('20110101','20201231')
df = pd.DataFrame(np.random.rand(len(dates),len(univ)),index=dates,columns=univ)
df

,AAPl,MSFT,BAC,GS
2011-01-01,0.400416,0.604771,0.646094,0.261449
2011-01-02,0.542022,0.459273,0.633180,0.556582
2011-01-03,0.192264,0.650311,0.959048,0.822497
2011-01-04,0.554892,0.987566,0.326609,0.031469
2011-01-05,0.389224,0.287320,0.310158,0.987517
...,...,...,...,...
2020-12-27,0.176294,0.050334,0.574050,0.247455
2020-12-28,0.967752,0.624912,0.281355,0.676943
2020-12-29,0.971693,0.576866,0.407887,0.634631
2020-12-30,0.576650,0.302471,0.556038,0.142821


In [7]:
sector = {'AAPl': 'Tech', 'MSFT':'Tech','BAC':'Finance', 'GS':'Finance'}

data = []
for dt in dates:
    for x in univ:
        data.append([x,sector[x],dt,np.random.rand(),np.random.rand()])
data = pd.DataFrame(data, columns=['ticker','sector','date','signal1','signal2'])
data

,ticker,sector,date,signal1,signal2
0,AAPl,Tech,2011-01-01,0.060459,0.626766
1,MSFT,Tech,2011-01-01,0.406203,0.612934
2,BAC,Finance,2011-01-01,0.885134,0.039667
3,GS,Finance,2011-01-01,0.389249,0.497699
4,AAPl,Tech,2011-01-02,0.258598,0.850239
...,...,...,...,...,...
14607,GS,Finance,2020-12-30,0.073031,0.896958
14608,AAPl,Tech,2020-12-31,0.553479,0.888660
14609,MSFT,Tech,2020-12-31,0.233022,0.645569
14610,BAC,Finance,2020-12-31,0.294393,0.799052


GroupBY

In [10]:
#what's the average of each signal of each ticker
data.groupby('ticker').mean(numeric_only=True)

,signal1,signal2
ticker,,
AAPl,0.504083,0.498148
BAC,0.500619,0.493258
GS,0.498485,0.504693
MSFT,0.495103,0.500774


In [14]:
#what's average signal of each sector on each day
data.groupby(['sector','date']).mean(numeric_only=True)

signal1   signal2
sector  date                          
Finance 2011-01-01  0.637191  0.268683
        2011-01-02  0.474555  0.291471
        2011-01-03  0.738140  0.417654
        2011-01-04  0.392258  0.698345
        2011-01-05  0.822754  0.669429
...                      ...       ...
Tech    2020-12-27  0.235037  0.660162
        2020-12-28  0.435918  0.651596
        2020-12-29  0.441396  0.647537
        2020-12-30  0.674314  0.552876
        2020-12-31  0.393250  0.767114

[7306 rows x 2 columns]

In [15]:
data.groupby(['sector','date'])[['signal1']].mean()

signal1
sector  date                
Finance 2011-01-01  0.637191
        2011-01-02  0.474555
        2011-01-03  0.738140
        2011-01-04  0.392258
        2011-01-05  0.822754
...                      ...
Tech    2020-12-27  0.235037
        2020-12-28  0.435918
        2020-12-29  0.441396
        2020-12-30  0.674314
        2020-12-31  0.393250

[7306 rows x 1 columns]

In [19]:
# use apply with groupby to pass an arbitrary function

def max_minus_min(x):
    return x.max()-x.min()
data.groupby(['sector','date'])[['signal1','signal2']].apply(max_minus_min)

signal1   signal2
sector  date                          
Finance 2011-01-01  0.495886  0.458032
        2011-01-02  0.694370  0.573256
        2011-01-03  0.121400  0.326998
        2011-01-04  0.024614  0.297517
        2011-01-05  0.240851  0.457272
...                      ...       ...
Tech    2020-12-27  0.336259  0.273776
        2020-12-28  0.063981  0.186156
        2020-12-29  0.733480  0.479684
        2020-12-30  0.323339  0.161335
        2020-12-31  0.320457  0.243091

[7306 rows x 2 columns]

In [20]:
#return a dataframe instead of a series
def demean(x):
    return x-x.mean()

data.groupby(['sector','date'])[['signal1','signal2']].apply(demean)


signal1   signal2
sector  date                                
Finance 2011-01-01 2      0.247943 -0.229016
                   3     -0.247943  0.229016
        2011-01-02 6      0.347185  0.286628
                   7     -0.347185 -0.286628
        2011-01-03 10    -0.060700 -0.163499
...                            ...       ...
Tech    2020-12-29 14601  0.366740 -0.239842
        2020-12-30 14604 -0.161670  0.080667
                   14605  0.161670 -0.080667
        2020-12-31 14608  0.160229  0.121546
                   14609 -0.160229 -0.121546

[14612 rows x 2 columns]

In [22]:
ser = data.groupby(['sector', 'date'])[['signal1','signal2']].mean()
df

,AAPl,MSFT,BAC,GS
2011-01-01,0.400416,0.604771,0.646094,0.261449
2011-01-02,0.542022,0.459273,0.633180,0.556582
2011-01-03,0.192264,0.650311,0.959048,0.822497
2011-01-04,0.554892,0.987566,0.326609,0.031469
2011-01-05,0.389224,0.287320,0.310158,0.987517
...,...,...,...,...
2020-12-27,0.176294,0.050334,0.574050,0.247455
2020-12-28,0.967752,0.624912,0.281355,0.676943
2020-12-29,0.971693,0.576866,0.407887,0.634631
2020-12-30,0.576650,0.302471,0.556038,0.142821


In [23]:
df.groupby(level=0).mean()

,AAPl,MSFT,BAC,GS
2011-01-01,0.400416,0.604771,0.646094,0.261449
2011-01-02,0.542022,0.459273,0.633180,0.556582
2011-01-03,0.192264,0.650311,0.959048,0.822497
2011-01-04,0.554892,0.987566,0.326609,0.031469
2011-01-05,0.389224,0.287320,0.310158,0.987517
...,...,...,...,...
2020-12-27,0.176294,0.050334,0.574050,0.247455
2020-12-28,0.967752,0.624912,0.281355,0.676943
2020-12-29,0.971693,0.576866,0.407887,0.634631
2020-12-30,0.576650,0.302471,0.556038,0.142821


In [24]:
month = [x.month for x in data['date']]

In [26]:
data.groupby(month).mean(numeric_only=True)

,signal1,signal2
1,0.511179,0.497541
2,0.486962,0.501926
3,0.510780,0.501421
4,0.502254,0.492071
5,0.510484,0.493292
6,0.504010,0.497730
7,0.501150,0.491430
8,0.500014,0.485999
9,0.487006,0.516893
10,0.496575,0.512032


In [27]:
for key,val in data.groupby('sector'):
    print(val)

      ticker   sector       date   signal1   signal2
2        BAC  Finance 2011-01-01  0.885134  0.039667
3         GS  Finance 2011-01-01  0.389249  0.497699
6        BAC  Finance 2011-01-02  0.821740  0.578099
7         GS  Finance 2011-01-02  0.127370  0.004843
10       BAC  Finance 2011-01-03  0.677440  0.254155
...      ...      ...        ...       ...       ...
14603     GS  Finance 2020-12-29  0.137489  0.267134
14606    BAC  Finance 2020-12-30  0.009493  0.194916
14607     GS  Finance 2020-12-30  0.073031  0.896958
14610    BAC  Finance 2020-12-31  0.294393  0.799052
14611     GS  Finance 2020-12-31  0.731425  0.755994

[7306 rows x 5 columns]
      ticker sector       date   signal1   signal2
0       AAPl   Tech 2011-01-01  0.060459  0.626766
1       MSFT   Tech 2011-01-01  0.406203  0.612934
4       AAPl   Tech 2011-01-02  0.258598  0.850239
5       MSFT   Tech 2011-01-02  0.028459  0.204082
8       AAPl   Tech 2011-01-03  0.135779  0.933042
...      ...    ...        ...   

Pivot

In [28]:
#put signal into a df with column tickers and rows dates
df1 = data.set_index(['date','ticker'])['signal1'].unstack(level=1)
df1

ticker,AAPl,BAC,GS,MSFT
date,,,,
2011-01-01,0.060459,0.885134,0.389249,0.406203
2011-01-02,0.258598,0.821740,0.127370,0.028459
2011-01-03,0.135779,0.677440,0.798840,0.297820
2011-01-04,0.612061,0.379951,0.404565,0.074699
2011-01-05,0.111927,0.943179,0.702328,0.715093
...,...,...,...,...
2020-12-27,0.066908,0.612615,0.653893,0.403167
2020-12-28,0.403928,0.590916,0.054874,0.467909
2020-12-29,0.074656,0.742348,0.137489,0.808136


In [29]:
df2 = data.pivot_table(index='date',columns='ticker',values='signal1')
df2

ticker,AAPl,BAC,GS,MSFT
date,,,,
2011-01-01,0.060459,0.885134,0.389249,0.406203
2011-01-02,0.258598,0.821740,0.127370,0.028459
2011-01-03,0.135779,0.677440,0.798840,0.297820
2011-01-04,0.612061,0.379951,0.404565,0.074699
2011-01-05,0.111927,0.943179,0.702328,0.715093
...,...,...,...,...
2020-12-27,0.066908,0.612615,0.653893,0.403167
2020-12-28,0.403928,0.590916,0.054874,0.467909
2020-12-29,0.074656,0.742348,0.137489,0.808136


In [30]:
df1.equals(df2)

True

In [31]:
data.pivot_table(index='date',columns='sector',values='signal1', aggfunc=np.mean)


C:\Users\vansh\AppData\Local\Temp\ipykernel_6428\384251995.py:1: FutureWarning: The provided callable <function mean at 0x0000019F6298C0E0> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  data.pivot_table(index='date',columns='sector',values='signal1', aggfunc=np.mean)


sector,Finance,Tech
date,,
2011-01-01,0.637191,0.233331
2011-01-02,0.474555,0.143529
2011-01-03,0.738140,0.216799
2011-01-04,0.392258,0.343380
2011-01-05,0.822754,0.413510
...,...,...
2020-12-27,0.633254,0.235037
2020-12-28,0.322895,0.435918
2020-12-29,0.439918,0.441396


In [32]:
data.pivot_table(index='date',columns='sector',values='signal1', aggfunc=np.max)

C:\Users\vansh\AppData\Local\Temp\ipykernel_6428\2243241853.py:1: FutureWarning: The provided callable <function max at 0x0000019F629876A0> is currently using DataFrameGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  data.pivot_table(index='date',columns='sector',values='signal1', aggfunc=np.max)


sector,Finance,Tech
date,,
2011-01-01,0.885134,0.406203
2011-01-02,0.821740,0.258598
2011-01-03,0.798840,0.297820
2011-01-04,0.404565,0.612061
2011-01-05,0.943179,0.715093
...,...,...
2020-12-27,0.653893,0.403167
2020-12-28,0.590916,0.467909
2020-12-29,0.742348,0.808136


Merge

In [33]:
data2 = []
for dt in dates:
    for x in ['Tech','Finance']:
        data2.append([x,dt,np.random.rand(),np.random.rand()])

data2 = pd.DataFrame(data2, columns = ['sector', 'date', 'signal3', 'signal4'])
data2

,sector,date,signal3,signal4
0,Tech,2011-01-01,0.376396,0.389454
1,Finance,2011-01-01,0.726882,0.415858
2,Tech,2011-01-02,0.212833,0.550786
3,Finance,2011-01-02,0.750847,0.760462
4,Tech,2011-01-03,0.061546,0.133444
...,...,...,...,...
7301,Finance,2020-12-29,0.603439,0.591361
7302,Tech,2020-12-30,0.729819,0.861638
7303,Finance,2020-12-30,0.227236,0.439774
7304,Tech,2020-12-31,0.096211,0.322499


In [34]:
data.merge(data2, left_on=['sector','date'],right_on=['sector','date'])

,ticker,sector,date,signal1,signal2,signal3,signal4
0,AAPl,Tech,2011-01-01,0.060459,0.626766,0.376396,0.389454
1,MSFT,Tech,2011-01-01,0.406203,0.612934,0.376396,0.389454
2,BAC,Finance,2011-01-01,0.885134,0.039667,0.726882,0.415858
3,GS,Finance,2011-01-01,0.389249,0.497699,0.726882,0.415858
4,AAPl,Tech,2011-01-02,0.258598,0.850239,0.212833,0.550786
...,...,...,...,...,...,...,...
14607,GS,Finance,2020-12-30,0.073031,0.896958,0.227236,0.439774
14608,AAPl,Tech,2020-12-31,0.553479,0.888660,0.096211,0.322499
14609,MSFT,Tech,2020-12-31,0.233022,0.645569,0.096211,0.322499
14610,BAC,Finance,2020-12-31,0.294393,0.799052,0.407708,0.518441
